In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.functions import expr, round, col, avg, format_number 
from pyspark.sql.types import DoubleType, StringType, StructField, StructType, IntegerType
from pyspark.sql.functions import col, sum, round, lit, concat, when, count, coalesce, upper, udf

warehouse_location = 'heducations://heducations-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = builder.getOrCreate() #spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark = builder.getOrCreate() #spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# read data from the silver tables
hiv = spark.table("projeto.Hiv_NY")

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, avg, format_number
from pyspark.sql.types import StringType

# Crie uma Spark session
spark = SparkSession.builder.appName("ExemploHIV").getOrCreate()

# Suponha que você tenha DataFrames chamados "hivF" e "hivM" com as modificações já feitas
# Certifique-se de ajustar os nomes das colunas conforme sua estrutura real de dados

# Função UDF para capitalizar a primeira letra de cada palavra
def initcap_udf(s):
    return ' '.join(word.capitalize() for word in s.split())

# Registrar a UDF no Spark
initcap_udf_spark = udf(initcap_udf, StringType())

# Aplicar a UDF às colunas Borough e Gender nos DataFrames
hiv = hiv.withColumn("Borough", initcap_udf_spark(col("Borough")))
hiv = hiv.withColumn("Gender", initcap_udf_spark(col("Gender")))

# Lista de bairros desejados
bairros_desejados = ["Bronx", "Brooklyn", "Manhattan", "Queens", "Staten Island"]

# Filtrar apenas as linhas desejadas para mulheres e homens de todas as idades nos DataFrames
filtro_f = (col("Gender") == "Female") & (col("Age") == "All") & (col("Borough").isin(bairros_desejados))
filtro_m = (col("Gender") == "Male") & (col("Age") == "All") & (col("Borough").isin(bairros_desejados))

dados_todas_idades_f = hiv.filter(filtro_f).groupBy('Borough').agg(format_number(avg('HIV_diagnosis_rate'), 2).alias('Female_HIV_diagnosis_rate'))
dados_todas_idades_m = hiv.filter(filtro_m).groupBy('Borough').agg(format_number(avg('HIV_diagnosis_rate'), 2).alias('Male_HIV_diagnosis_rate'))

# Juntar os resultados das duas consultas usando o bairro como chave
resultado_final = dados_todas_idades_f.join(dados_todas_idades_m, on='Borough')

# Calcular a diferença entre as taxas masculinas e femininas
resultado_final = resultado_final.withColumn('Gender_Difference_HIV_diagnosis_rate', col('Male_HIV_diagnosis_rate') - col('Female_HIV_diagnosis_rate'))

# Adicionar as colunas 'State' e 'Country' no início
resultado_final = resultado_final.withColumn("State", lit("New York")).withColumn("Country", lit("United States"))

# Reorganizar a ordem das colunas
resultado_final = resultado_final.select("State", "Country", "Borough", "Female_HIV_diagnosis_rate", "Male_HIV_diagnosis_rate", "Gender_Difference_HIV_diagnosis_rate")

# Exibir os resultados
resultado_final.show()


+--------+-------------+-------------+-------------------------+-----------------------+------------------------------------+
|   State|      Country|      Borough|Female_HIV_diagnosis_rate|Male_HIV_diagnosis_rate|Gender_Difference_HIV_diagnosis_rate|
+--------+-------------+-------------+-------------------------+-----------------------+------------------------------------+
|New York|United States|       Queens|                     7.80|                  36.32|                               28.52|
|New York|United States|     Brooklyn|                    14.57|                  60.15|                               45.58|
|New York|United States|Staten Island|                     8.79|                  24.31|                               15.52|
|New York|United States|    Manhattan|                    18.81|                 104.22|                               85.41|
|New York|United States|        Bronx|                    25.14|                  66.58|                              

In [4]:
customSchema = StructType([
    StructField("Borough", StringType(), True),        
    StructField("Female_HIV_diagnosis_rate", DoubleType(), True),
    StructField("Male_HIV_diagnosis_rate", DoubleType(), True),
    StructField("Gender_Difference_HIV_diagnosis_rate", DoubleType(), True),        
])

hiv \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("hdfs://hdfs-nn:9000/warehouse/projeto_gold.db/Gender_Difference_Hiv_NY")